#**PySpark Installation**

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
#!wget https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz

--2021-12-08 16:05:09--  https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.95.219, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272668489 (260M) [application/x-gzip]
Saving to: ‘spark-3.2.0-bin-hadoop2.7.tgz’

spark-3.2.0-bin-had 100%[===================>] 260.04M  26.2MB/s    in 11s     

2021-12-08 16:05:20 (24.2 MB/s) - ‘spark-3.2.0-bin-hadoop2.7.tgz’ saved [272668489/272668489]



In [ ]:
#!tar -xvzf /content/spark-3.2.0-bin-hadoop2.7.tgz

spark-3.2.0-bin-hadoop2.7/
spark-3.2.0-bin-hadoop2.7/NOTICE
spark-3.2.0-bin-hadoop2.7/kubernetes/
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/python_executable_check.py
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/autoscale.py
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/worker_memory_check.py
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/py_container_checks.py
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/decommissioning.py
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/pyfiles.py
spark-3.2.0-bin-hadoop2.7/kubernetes/tests/decommissioning_cleanup.py
spark-3.2.0-bin-hadoop2.7/kubernetes/dockerfiles/
spark-3.2.0-bin-hadoop2.7/kubernetes/dockerfiles/spark/
spark-3.2.0-bin-hadoop2.7/kubernetes/dockerfiles/spark/decom.sh
spark-3.2.0-bin-hadoop2.7/kubernetes/dockerfiles/spark/entrypoint.sh
spark-3.2.0-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/
spark-3.2.0-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/R/
spark-3.2.0-bin-hadoop2.7/kubernetes/docker

In [ ]:
#!pip install -q findspark

In [ ]:
#import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"

In [ ]:
#import findspark
#findspark.init()

In [ ]:
#findspark.find()

'/content/spark-3.2.0-bin-hadoop2.7'

In [ ]:
#from pyspark.sql import SparkSession
'''
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark
'''

In [ ]:
from pyspark.sql.types import IntegerType,StringType
from pyspark.sql.functions import col
from pyspark.ml.feature import RegexTokenizer,StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.ml.classification import LogisticRegression, NaiveBayes, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline

#**Data Preparation**

##**For Instagram**

### **Apple Data**

In [ ]:
# Data from the apple app store
instagram_apple=spark.read.format('csv').load('/content/data/instagram_apple.csv',header=True)
instagram_apple.show()

+--------------------+--------------------+--------------------+-------------------+---------------+--------------------+--------------------+
|                 _c0|              rating|            isEdited|               date|       userName|               title|              review|
+--------------------+--------------------+--------------------+-------------------+---------------+--------------------+--------------------+
|                   0|                   5|               False|2020-12-23 07:09:20|       babyaggu|Stop promoting re...|I get the fact th...|
|                   1|                   4|               False|2020-10-20 05:00:26|        tanu_99|Action blocked ( ...|These is 4th time...|
|          Thank you"|                null|                null|               null|           null|                null|                null|
|                   2|                   5|               False|2021-01-08 05:35:46|   _arinsingh01|30 sec length opt...|Hi there , Instag...|

In [ ]:
# Data Cleaning 

# Removing the unwanted data 
cols=['_c0','isEdited','date','userName','title']
instagram_apple=instagram_apple.drop(*cols)
instagram_apple=instagram_apple.withColumn('rating',col('rating').cast(IntegerType()))
instagram_apple=instagram_apple.na.drop()

instagram_apple.show()

+------+--------------------+
|rating|              review|
+------+--------------------+
|     5|I get the fact th...|
|     4|These is 4th time...|
|     5|Hi there , Instag...|
|     3|"No doubt instagr...|
|     1|"Dear Instagram i...|
|     1|Please instagram ...|
|     5|Hello....I would ...|
|     3|     Dear Instagram,|
|     5|After reporting a...|
|     5|I’ve been blocked...|
|     1|After Tiktok got ...|
|     5|I am using Instag...|
|     1|Dear instagram it...|
|     5|I have been repor...|
|     5|For some reason e...|
|     3|I’ve been on Inst...|
|     1|I have been using...|
|     1|Dear INSTAGRAM, h...|
|     3|Dear Instagram I ...|
|     4|I’ve been using I...|
+------+--------------------+
only showing top 20 rows



### **Google Play Data** 

In [ ]:
# Data from google play store
instagram_google=spark.read.format('csv').load('/content/data/instagram_google.csv',header=True)
instagram_google.show()

+---+--------------------+------------------+--------------------+--------------------+-----+-------------+--------------------+-------------------+------------+---------+
|_c0|            reviewId|          userName|           userImage|             content|score|thumbsUpCount|reviewCreatedVersion|                 at|replyContent|repliedAt|
+---+--------------------+------------------+--------------------+--------------------+-----+-------------+--------------------+-------------------+------------+---------+
|  0|gp:AOqpTOFB-ewH6f...|           Radhika|https://play-lh.g...|My Insta account ...|    1|           68|      214.0.0.27.120|2021-11-17 20:29:19|        null|     null|
|  1|gp:AOqpTOFRAtLjU7...|   mayank upadhyay|https://play-lh.g...|I would give it o...|    1|          124|      214.1.0.29.120|2021-11-20 12:54:42|        null|     null|
|  2|gp:AOqpTOENgkPhHQ...|      Nihar Zaveri|https://play-lh.g...|Dear instagram......|    5|           78|      214.1.0.29.120|2021-12-04 1

In [ ]:
# Data Cleaning
cols=['_c0','reviewId','userName','userImage','thumbsUpCount','reviewCreatedVersion','','at','replyContent','repliedAt']
instagram_google=instagram_google.drop(*cols)

# Renaming the columns to create a common review-rating system 
instagram_google=instagram_google.withColumnRenamed('content','review')
instagram_google=instagram_google.withColumnRenamed('score','rating')

# Dropping all the null records
instagram_google=instagram_google.withColumn('rating',col('rating').cast(IntegerType()))
instagram_google=instagram_google.na.drop()

instagram_google.show()

+--------------------+------+
|              review|rating|
+--------------------+------+
|My Insta account ...|     1|
|I would give it o...|     1|
|Dear instagram......|     5|
|Lot of internet c...|     1|
|I don't know why ...|     1|
|It's been 3 4 wee...|     1|
|Hey instagram! I ...|     5|
|What is happening...|     1|
|Everything seems ...|     4|
|I would give it a...|     1|
|My account doesn'...|     1|
|"Could not see an...|     1|
|Having very bad e...|     1|
|At night I sent a...|     2|
|For some days , l...|     1|
|What I mean to sa...|     1|
|Please clear some...|     3|
|App is definitely...|     3|
|Recently instagra...|     2|
|Please add multi ...|     3|
+--------------------+------+
only showing top 20 rows



## **For Twitter**

### **Apple App Store Data**

In [ ]:
twitter_apple=spark.read.format('csv').load('/content/data/twitter_apple.csv',header=True)
twitter_apple.show()

+--------------------+--------------------+------+--------------------+----------------+-------------------+--------------------+
|                 _c0|            isEdited|rating|               title|        userName|               date|              review|
+--------------------+--------------------+------+--------------------+----------------+-------------------+--------------------+
|                   0|               False|     5|   Some improvements|       abhinav.s|2020-04-08 14:51:18|Could do a better...|
|                   1|               False|     3|Recommandation/Su...|       Chiru1108|2020-02-03 17:44:35|Your system is re...|
|               Still| your thing is be...|  null|                null|            null|               null|                null|
|                   2|               False|     1|Increase the word...|    BikramSaluja|2020-11-25 07:52:00|You need to incre...|
|                   3|               False|     5|  Universal Twitter.|    speakinghill|20

In [ ]:
# Data Cleaning 

# Removing the unwanted data 
cols=['_c0','isEdited','date','userName','title']
twitter_apple=twitter_apple.drop(*cols)
twitter_apple=twitter_apple.withColumn('rating',col('rating').cast(IntegerType()))
twitter_apple=twitter_apple.na.drop()

twitter_apple.show()

+------+--------------------+
|rating|              review|
+------+--------------------+
|     5|Could do a better...|
|     3|Your system is re...|
|     1|You need to incre...|
|     5|I have always won...|
|     5|This app doesn't ...|
|     4|Twitter help me s...|
|     3|You removed many ...|
|     5|Twitter is one so...|
|     4|I am using twitte...|
|     5|Twitter is a non ...|
|     4|Well I foolishly ...|
|     3|If you have ever ...|
|     1|The constant nudg...|
|     5|Great app as it w...|
|     5|connects to the l...|
|     5|    Why it’s needed?|
|     4|I love using twit...|
|     3|So I have multipl...|
|     5|Twitter is an amz...|
|     1|I wonder why the ...|
+------+--------------------+
only showing top 20 rows



### **Google Play Store Data**

In [ ]:
twitter_google=spark.read.format('csv').load('/content/data/twitter_google.csv',header=True)
twitter_google.show()

+---+--------------------+--------------------+--------------------+--------------------+-----+-------------+--------------------+-------------------+------------+---------+
|_c0|            reviewId|            userName|           userImage|             content|score|thumbsUpCount|reviewCreatedVersion|                 at|replyContent|repliedAt|
+---+--------------------+--------------------+--------------------+--------------------+-----+-------------+--------------------+-------------------+------------+---------+
|  0|gp:AOqpTOFy06BeUe...|    kimberly stewart|https://play-lh.g...|There are too man...|    2|          179|   9.20.0-release.00|2021-11-26 14:35:46|        null|     null|
|  1|gp:AOqpTOEvAgHkgi...|      Izuku Midoriya|https://play-lh.g...|Twitter please fi...|    1|          412|   9.20.0-release.00|2021-11-20 11:21:30|        null|     null|
|  2|gp:AOqpTOGwYagIau...|        Rachel Baker|https://play-lh.g...|"Since the last u...|    2|          166|   9.20.0-release.00|

In [ ]:
# Data Cleaning 

# Removing unwanted data
cols=['_c0','reviewId','userName','userImage','thumbsUpCount','reviewCreatedVersion','','at','replyContent','repliedAt']
twitter_google=twitter_google.drop(*cols)

# Renaming the columns to create a common review-rating system 
twitter_google=twitter_google.withColumnRenamed('content','review')
twitter_google=twitter_google.withColumnRenamed('score','rating')

# Removing the null records
twitter_google=twitter_google.withColumn('rating',col('rating').cast(IntegerType()))
twitter_google=twitter_google.na.drop()

twitter_google.show()

+--------------------+------+
|              review|rating|
+--------------------+------+
|There are too man...|     2|
|Twitter please fi...|     1|
|"Since the last u...|     2|
|Can't post replie...|     1|
|notification feed...|     1|
|This app and the ...|     4|
|Hi, I just want t...|     1|
|the new layout lo...|     1|
|"Imo it's totally...|     3|
|"Pretty good app ...|     4|
|Cant see all my n...|     2|
|Please add the co...|     4|
|It could desperat...|     5|
|I don't usually d...|     3|
|I dont know what ...|     1|
|Can't switch from...|     1|
|It currently isn'...|     1|
|Needs some refine...|     4|
|Notifications tab...|     1|
|This app now make...|     1|
+--------------------+------+
only showing top 20 rows



## **For Koo**

### **Apple App Store Data**

In [ ]:
koo_apple=spark.read.format('csv').load('/content/data/koo_apple.csv',header=True)
koo_apple.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 _c0|            userName|                date|              rating|              review|               title|            isEdited|   developerResponse|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   0|         ABWolverine|    10-02-2021 18:45|                   4|The initial roll ...|                null|                null|                null|
|Given the user ba...| glitches are exp...|                null|                null|                null|                null|                null|                null|
|         Some search| tag and syntax i...|                null|                null|                null|                null|                null|  

In [ ]:
# Data Cleaning

# Removing the unwanted data 
cols=['_c0','isEdited','date','userName','title','developerResponse']
koo_apple=koo_apple.drop(*cols)
koo_apple=koo_apple.withColumn('rating',col('rating').cast(IntegerType()))
koo_apple=koo_apple.dropna()

koo_apple.show()

+------+--------------------+
|rating|              review|
+------+--------------------+
|     4|The initial roll ...|
|     1|                  Hi|
|     4|Will directly com...|
|     5|Much Love ❤️ and ...|
|     5|Terrific platform...|
|     5|This is my review...|
|     4|Why it’s white in...|
|     4|Everything is goi...|
|     5|Improvements on t...|
|     5|I saw the version...|
|     1|Hi , I got to kno...|
|     5|First of all, kud...|
|     5|I have downloaded...|
|     5|The user interfac...|
|     5|Hey, I would like...|
|     4|People may say th...|
|     4|Although I have g...|
|     1|Thought this woul...|
|     5|In true sense koo...|
|     5|In case of same o...|
+------+--------------------+
only showing top 20 rows



### **Google Play Store Data**

In [ ]:
koo_google=spark.read.format('csv').load('/content/data/koo_google.csv',header=True)
koo_google.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+-----+-------------+--------------------+-------------------+--------------------+-------------------+
|                 _c0|            reviewId|            userName|           userImage|             content|score|thumbsUpCount|reviewCreatedVersion|                 at|        replyContent|          repliedAt|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----+-------------+--------------------+-------------------+--------------------+-------------------+
|                   0|gp:AOqpTOE9g8_pNS...|                  JS|https://play-lh.g...|5 Star !! Brillia...|    5|           73|              0.93.6|2021-09-08 12:49:02|Hey! We have deve...|               null|
|-Your Friends at ...| 2021-07-29 17:32:46|                null|                null|                null| null|         null|                null|               nu

In [ ]:
# Data Cleaning

# Removing unwanted column
cols=['_c0','reviewId','userName','userImage','thumbsUpCount','reviewCreatedVersion','','at','replyContent','repliedAt']
koo_google=koo_google.drop(*cols)

# Renaming the columns to create a common review-rating system 
koo_google=koo_google.withColumnRenamed('content','review')
koo_google=koo_google.withColumnRenamed('score','rating')

# Dropping all the null records
koo_google=koo_google.withColumn('rating',col('rating').cast(IntegerType()))
koo_google=koo_google.na.drop()

koo_google.show()

+--------------------+------+
|              review|rating|
+--------------------+------+
|5 Star !! Brillia...|     5|
|The app is very g...|     4|
|I'm loving it, no...|     5|
|Very good app but...|     5|
|I can speak - tha...|     5|
|One of india's be...|     5|
|Great Application...|     5|
|It's really slow!...|     1|
|The app developme...|     3|
|It's a great app....|     5|
|I am glad to see ...|     5|
|The app is still ...|     5|
|Enjoy the experie...|     5|
|Couple of issues....|     5|
|Please launch pas...|     5|
|Ability to showca...|     5|
|Fairly ok but the...|     5|
|Getting logged ou...|     5|
|This is a very go...|     5|
|More features sho...|     4|
+--------------------+------+
only showing top 20 rows



# **Basic Ranking**

In [ ]:
def basic_app_ranking(data1,data2):
  rank1=0
  rank2=0

  for i in range(1,6):
    current=data1.filter(data1['rating']==i).count()
    rank1=rank1+(current*i)
    current=data2.filter(data2['rating']==i).count()
    rank2=rank2+(current*i)

  rank1=rank1/data1.count()
  rank2=rank2/data2.count()

  final_rank=(rank1+rank2)/2

  return final_rank 

In [ ]:
instagram_rank=basic_app_ranking(instagram_apple,instagram_google)
print(instagram_rank)

2.830576923076923


In [ ]:
twitter_rank=basic_app_ranking(twitter_apple,twitter_google)
print(twitter_rank)

3.129585492227979


In [ ]:
koo_rank=basic_app_ranking(koo_apple,koo_google)
print(koo_rank)

4.192703045685279


In [ ]:
basic_rank_table=spark.createDataFrame(data=[['Instagram',instagram_rank],['Twitter',twitter_rank],['Koo',koo_rank]])
basic_rank_table=basic_rank_table.withColumnRenamed('_1','App')
basic_rank_table=basic_rank_table.withColumnRenamed('_2','Rank')
basic_rank_table=basic_rank_table.orderBy(col('Rank').desc())
basic_rank_table.show()

+---------+-----------------+
|      App|             Rank|
+---------+-----------------+
|      Koo|4.192703045685279|
|  Twitter|3.129585492227979|
|Instagram|2.830576923076923|
+---------+-----------------+



# **NLP**

In [ ]:
def nlp_dataset_creation(data):

  data=data.withColumn('rating',col('rating').cast(StringType()))

  # Tokenizing using Regular Expression
  tokenize=RegexTokenizer(inputCol='review',outputCol='words',pattern='\w')

  # Removing the stop words
  stop_words=['a','the','an','is','at','on','in'] #'.','!','$','%','&','(',')',',',,' '
  remove_stop_words=StopWordsRemover(inputCol='words',outputCol='filteredWords')

  # Using TFID Vectorizer
  tf=HashingTF(inputCol='filteredWords',outputCol='rawFeatures')
  idf=IDF(inputCol='rawFeatures',outputCol='features')

  # Creating Labels
  label=StringIndexer(inputCol='rating',outputCol='label')

  # Pipeline
  pipeline=Pipeline(stages=[tokenize,remove_stop_words,tf,idf,label])
  fit=pipeline.fit(data)
  dataset=fit.transform(data)
  return dataset

In [ ]:
nlp_dataset=nlp_dataset_creation(instagram_google)
nlp_dataset.show()

+--------------------+------+--------------------+--------------------+--------------------+--------------------+-----+
|              review|rating|               words|       filteredWords|         rawFeatures|            features|label|
+--------------------+------+--------------------+--------------------+--------------------+--------------------+-----+
|My Insta account ...|     1|[ ,  ,  ,  ,  ,  ...|[ ,  ,  ,  ,  ,  ...|(262144,[45404,14...|(262144,[45404,14...|  0.0|
|I would give it o...|     1|[ ,  ,  ,  ,  , ....|[ ,  ,  ,  ,  , ....|(262144,[45404,98...|(262144,[45404,98...|  0.0|
|Dear instagram......|     5|[ , ... ,  ,  ,  ...|[ , ... ,  ,  ,  ...|(262144,[146416,1...|(262144,[146416,1...|  3.0|
|Lot of internet c...|     1|[ ,  ,  ,  , , , ...|[ ,  ,  ,  , , , ...|(262144,[38640,45...|(262144,[38640,45...|  0.0|
|I don't know why ...|     1|[ , ',  ,  ,  ,  ...|[ , ',  ,  ,  ,  ...|(262144,[45404,14...|(262144,[45404,14...|  0.0|
|It's been 3 4 wee...|     1|[',  ,  ,  

In [ ]:
# Data Split
(training_data,testing_data)=nlp_dataset.randomSplit([0.8,0.2],seed=100)
training_data.show()

+--------------------+------+--------------------+--------------------+--------------------+--------------------+-----+
|              review|rating|               words|       filteredWords|         rawFeatures|            features|label|
+--------------------+------+--------------------+--------------------+--------------------+--------------------+-----+
|"Could not see an...|     1|[",  ,  ,  ,  ,  ...|[",  ,  ,  ,  ,  ...|(262144,[90318,13...|(262144,[90318,13...|  0.0|
|"From last few DA...|     2|[",  ,  ,  , ! , ...|[",  ,  ,  , ! , ...|(262144,[40306,44...|(262144,[40306,44...|  1.0|
|"I am facing ""Co...|     4|[",  ,  ,  "",  ,...|[",  ,  ,  "",  ,...|(262144,[90318,13...|(262144,[90318,13...|  4.0|
|"I have been tryi...|     1|[",  ,  ,  ,  ,  ...|[",  ,  ,  ,  ,  ...|(262144,[77345,14...|(262144,[77345,14...|  0.0|
|"I have faced pro...|     1|[",  ,  ,  ,  ,  ...|[",  ,  ,  ,  ,  ...|(262144,[2491,614...|(262144,[2491,614...|  0.0|
|"Its showing ""Er...|     1|[",  ,  "",

In [ ]:
# Logistic Regression Model
logReg=LogisticRegression(maxIter=20,elasticNetParam=0,regParam=0.3)
lrModel=logReg.fit(training_data)

In [ ]:
predictions=lrModel.transform(testing_data)
predictions=predictions.select('review','features','rating','label','probability','prediction')
predictions.show()

NameError: ignored

In [ ]:
evaluator=MulticlassClassificationEvaluator(predictionCol='prediction')
evaluator.evaluate(predictions)

0.41367521367521365

In [ ]:
# Naive Bayes 
nbc=NaiveBayes(smoothing=1)
nbModel=nbc.fit(training_data)
predictions=nbModel.transform(testing_data)
predictions=predictions.select('review','features','rating','label','probability','prediction')
predictions.show()

+--------------------+--------------------+------+-----+--------------------+----------+
|              review|            features|rating|label|         probability|prediction|
+--------------------+--------------------+------+-----+--------------------+----------+
|"I gave it only o...|(262144,[90318,14...|     1|  0.0|[0.99999997330316...|       0.0|
|"my review keeps ...|(262144,[10749,77...|     1|  0.0|[0.99999999999985...|       0.0|
|All my Instagram ...|(262144,[116338,1...|     4|  4.0|[0.99999929020560...|       0.0|
|At the Beginning ...|(262144,[45404,14...|     2|  1.0|[0.98614639804681...|       0.0|
|Couldn't refresh ...|(262144,[45404,10...|     1|  0.0|[0.99968009178754...|       0.0|
|Everything is gre...|(262144,[146416],...|     4|  4.0|[0.55900621118012...|       0.0|
|For some days , l...|(262144,[21308,98...|     1|  0.0|[0.99996964764439...|       0.0|
|For the past 15 d...|(262144,[45404,14...|     1|  0.0|[0.93245838257327...|       0.0|
|Hate this new upd...

In [ ]:
evaluator=MulticlassClassificationEvaluator( predictionCol='prediction')
evaluator.evaluate(predictions)

0.38244241634072146

In [ ]:
rf=RandomForestClassifier(featuresCol='features',labelCol='label',numTrees=50)
rfModel=rf.fit(training_data)
predictions=rfModel.transform(testing_data)

In [ ]:
predictions=lrModel.transform(testing_data)
predictions=predictions.select('review','features','rating','label','probability','prediction')
predictions.show()

+--------------------+--------------------+------+-----+--------------------+----------+
|              review|            features|rating|label|         probability|prediction|
+--------------------+--------------------+------+-----+--------------------+----------+
|"I gave it only o...|(262144,[90318,14...|     1|  0.0|[0.78388101786509...|       0.0|
|"my review keeps ...|(262144,[10749,77...|     1|  0.0|[0.89162004229612...|       0.0|
|All my Instagram ...|(262144,[116338,1...|     4|  4.0|[0.72918960786686...|       0.0|
|At the Beginning ...|(262144,[45404,14...|     2|  1.0|[0.59740846242115...|       0.0|
|Couldn't refresh ...|(262144,[45404,10...|     1|  0.0|[0.62584830915321...|       0.0|
|Everything is gre...|(262144,[146416],...|     4|  4.0|[0.56798116132667...|       0.0|
|For some days , l...|(262144,[21308,98...|     1|  0.0|[0.61346292770563...|       0.0|
|For the past 15 d...|(262144,[45404,14...|     1|  0.0|[0.60224650951381...|       0.0|
|Hate this new upd...

In [ ]:
evaluator=MulticlassClassificationEvaluator( predictionCol='prediction')
evaluator.evaluate(predictions)

0.41367521367521365